# Interstate Commerce Network Analysis

**Research Questions:**
1. Which states are most influential in our network of interstate flows?
2. What are the most significant flows between those states?

**Data Source:** U.S. Census Bureau CFS 2017 Public Use File

**Note:** This notebook downloads official Census data and uses validated analysis scripts to reproduce research findings.

## Step 1: Setup - Download CFS Data from Census Bureau

Download the official CFS 2017 Public Use File directly from the U.S. Census Bureau (~140MB, takes 2-3 minutes).

In [ ]:
# Download CFS 2017 data from Census Bureau
import os
import urllib.request
import zipfile

# Census Bureau official data URL
data_url = "https://www2.census.gov/programs-surveys/cfs/datasets/2017/CFS%202017%20PUF%20CSV.zip"

# Download if not already present
if not os.path.exists('cfs_2017_puf.csv'):
    print("Downloading CFS 2017 data from Census Bureau...")
    urllib.request.urlretrieve(data_url, 'cfs_data.zip')
    
    print("Extracting data...")
    with zipfile.ZipFile('cfs_data.zip', 'r') as zip_ref:
        zip_ref.extractall('.')
    
    # Rename the extracted file to match what our scripts expect
    if os.path.exists('CFS 2017 PUF CSV.csv'):
        os.rename('CFS 2017 PUF CSV.csv', 'cfs_2017_puf.csv')
        print("Renamed file to cfs_2017_puf.csv")
    
    # Clean up zip file
    os.remove('cfs_data.zip')
    print("Data downloaded successfully!")
else:
    print("CFS data already present.")

## Step 2: Import Analysis Scripts from GitHub

Download our validated analysis scripts from the GitHub repository.

In [ ]:
# Download analysis scripts from GitHub
import urllib.request

# GitHub raw file URLs
base_url = "https://raw.githubusercontent.com/rsthornton/cfs-network-analysis/main/analysis/"

scripts = [
    "centrality_analysis.py",
    "flow_extraction.py"
]

for script in scripts:
    if not os.path.exists(script):
        print(f"Downloading {script}...")
        urllib.request.urlretrieve(base_url + script, script)
        print(f"  ✓ {script} downloaded")
    else:
        print(f"  ✓ {script} already present")

print("\nAnalysis scripts ready!")

## Step 3: Install Required Libraries

Install the necessary Python packages for the analysis.

In [ ]:
# Install required packages
!pip install pandas networkx matplotlib seaborn -q

print("Required libraries installed.")

## Step 4: Run Three-Level Centrality Analysis

Identify the most influential states using our three-level framework:
- **MACRO**: Regional bridging power (betweenness centrality)
- **MESO**: Influence networks (eigenvector centrality)
- **MICRO**: Distribution power (weighted out-degree)

In [ ]:
# Run centrality analysis
!python centrality_analysis.py --data cfs_2017_puf.csv --top-n 10

print("\nCentrality analysis complete!")

## Step 5: Load and Display Centrality Results

Load the results and create a simple summary table showing the most influential states.

In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt

# Check if results directory exists
if not os.path.exists('results'):
    print("ERROR: Results directory not found. Please run Step 4 first.")
else:
    # Load centrality results
    results_files = os.listdir('results')
    json_files = [f for f in results_files if f.startswith('centrality_analysis') and f.endswith('.json')]
    
    if not json_files:
        print("ERROR: No centrality analysis results found. Please run Step 4 first.")
    else:
        json_file = json_files[0]
        
        with open(f'results/{json_file}', 'r') as f:
            results = json.load(f)
        
        # Create summary table of top 5 states at each level
        print("=" * 60)
        print("MOST INFLUENTIAL STATES BY LEVEL (Top 5)")
        print("=" * 60)
        
        levels = [
            ('MACRO - Regional Bridging', 'macro_level'),
            ('MESO - Influence Networks', 'meso_level'),
            ('MICRO - Distribution Power', 'micro_level')
        ]
        
        for level_name, level_key in levels:
            print(f"\n{level_name}:")
            leaders = results['three_level_analysis'][level_key]['leaders'][:5]
            for i, state_data in enumerate(leaders, 1):
                # state_data is a list: [state_id, state_code, score]
                state_code = state_data[1]
                score = state_data[2]
                print(f"  {i}. {state_code}: {score:.3f}")
        
        # Show multi-level leaders
        print("\n" + "=" * 60)
        print("MULTI-LEVEL LEADERS (States appearing in multiple rankings)")
        print("=" * 60)
        for state in results['three_level_analysis']['multi_level_leaders'][:5]:
            print(f"  {state['state_name']}: {state['level_count']} levels - Score: {state['total_score']:.2f}")
        
        # Create visualization showing key finding: TX, CA, NY as top 3 influential states
        print("\n" + "=" * 60)
        print("KEY FINDING: Most Influential States in Network")
        print("=" * 60)
        
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # Show MESO level (Influence Networks) - the key research finding
        meso_leaders = results['three_level_analysis']['meso_level']['leaders'][:10]
        states = [leader[1] for leader in meso_leaders]
        scores = [leader[2] for leader in meso_leaders]
        
        # Highlight top 3 (TX, CA, NY) in different colors
        colors = ['#d73027' if i < 3 else 'steelblue' for i in range(len(states))]
        
        ax.bar(states, scores, color=colors)
        ax.set_xlabel('State', fontsize=12)
        ax.set_ylabel('Eigenvector Centrality Score', fontsize=12)
        ax.set_title('Most Influential States in Interstate Commerce Network\n(Top 3: TX, CA, NY)', fontsize=14)
        ax.grid(True, alpha=0.3, axis='y')
        
        # Add text annotation
        ax.text(0.02, 0.95, 'Red bars: Top 3 most influential states', 
                transform=ax.transAxes, fontsize=10, 
                bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgray"))
        
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
        
        print(f"\n✓ Key Finding Confirmed: TX, CA, NY are the top 3 most influential states")
        print(f"  in interstate commerce networks (eigenvector centrality)")

## Step 6: Extract Bilateral Flows Between Top States

Analyze the most significant commodity flows between the influential states identified above.

In [ ]:
# Extract flows between the top 3 most influential states: TX, CA, NY
if 'results' not in locals():
    print("ERROR: Please run Step 5 first to load centrality results.")
else:
    # Focus on the top 3 most influential states from MESO level
    meso_leaders = results['three_level_analysis']['meso_level']['leaders'][:3]
    top_3_states = [leader[1] for leader in meso_leaders]  # TX, CA, NY
    
    states_list = ','.join(sorted(top_3_states))
    print(f"Analyzing specific commodity flows between top 3 influential states: {states_list}")
    
    # Run flow extraction focusing only on TX, CA, NY
    !python flow_extraction.py --data cfs_2017_puf.csv --states {states_list} --top-n 25

## Step 7: Display Flow Analysis Results

Show the most significant bilateral flows between influential states.

In [ ]:
# Display specific commodity flows between TX, CA, NY
if not os.path.exists('results'):
    print("ERROR: Results directory not found. Please run Step 6 first.")
else:
    flow_files = os.listdir('results')
    csv_files = [f for f in flow_files if f.startswith('bilateral_flows') and f.endswith('.csv')]
    
    if not csv_files:
        print("ERROR: No bilateral flow results found. Please run Step 6 first.")
    else:
        csv_file = csv_files[0]
        
        flows_df = pd.read_csv(f'results/{csv_file}')
        
        # Display top flows with commodity details
        print("=" * 80)
        print("TOP COMMODITY FLOWS BETWEEN MOST INFLUENTIAL STATES (TX, CA, NY)")
        print("=" * 80)
        print("\nRank | Route        | Commodity | Value ($B) | Weight (M tons)")
        print("-" * 80)
        
        for idx, row in flows_df.head(15).iterrows():
            commodity = row['commodity_name'] if 'commodity_name' in row.index else f"SCTG-{row['SCTG']}"
            print(f"{idx+1:4d} | {row['orig_state_name']:^2} → {row['dest_state_name']:^2} | {commodity:^15} | "
                  f"${row['weighted_value']/1e9:8.2f} | {row['weighted_tons']/1e6:8.2f}")
        
        # Summary by route
        print("\n" + "=" * 80)
        print("SUMMARY BY ROUTE")
        print("=" * 80)
        route_summary = flows_df.groupby(['orig_state_name', 'dest_state_name']).agg({
            'weighted_value': 'sum',
            'weighted_tons': 'sum'
        }).round(2)
        
        for (origin, dest), data in route_summary.iterrows():
            print(f"{origin} → {dest}: ${data['weighted_value']/1e9:.1f}B ({data['weighted_tons']/1e6:.1f}M tons)")
        
        # Top commodities overall
        print("\n" + "=" * 80)
        print("TOP COMMODITIES IN TX-CA-NY TRIANGLE")
        print("=" * 80)
        commodity_summary = flows_df.groupby('SCTG').agg({
            'weighted_value': 'sum',
            'commodity_name': 'first'
        }).sort_values('weighted_value', ascending=False).head(5)
        
        for sctg, data in commodity_summary.iterrows():
            commodity = data['commodity_name'] if 'commodity_name' in data.index else f"SCTG-{sctg}"
            print(f"  {commodity}: ${data['weighted_value']/1e9:.1f}B")
        
        print(f"\nTotal trade volume between TX, CA, NY: ${flows_df['weighted_value'].sum()/1e9:.1f} billion")

## Step 8: Network Visualization

Generate a simple network diagram showing trade flows between the most influential states.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

# Check if results are loaded
if 'results' not in locals():
    print("ERROR: Please run Step 5 first to load centrality results.")
else:
    # Check if flow data exists
    if not os.path.exists('results'):
        print("ERROR: Please run Step 6 first to generate flow data.")
    else:
        flow_files = os.listdir('results')
        csv_files = [f for f in flow_files if f.startswith('bilateral_flows') and f.endswith('.csv')]
        
        if not csv_files:
            print("ERROR: No flow data found. Please run Step 6 first.")
        else:
            # SCTG commodity mapping
            SCTG_CODES = {
                1: 'Animals and Fish (live)', 2: 'Cereal Grains', 3: 'Agricultural Products', 
                4: 'Animal Feed and Products', 5: 'Meat, Poultry, Fish, Seafood',
                6: 'Milled Grain Products and Bakery', 7: 'Other Prepared Foodstuffs', 
                8: 'Alcoholic Beverages', 9: 'Tobacco Products',
                10: 'Monumental or Building Stone', 11: 'Natural Sands', 12: 'Gravel and Crushed Stone',
                13: 'Other Non-Metallic Minerals', 14: 'Metallic Ores and Concentrates',
                15: 'Coal', 16: 'Crude Petroleum', 17: 'Gasoline and Aviation Fuel', 
                18: 'Fuel Oils', 19: 'Other Coal and Petroleum Products',
                20: 'Basic Chemicals', 21: 'Pharmaceutical Products', 22: 'Fertilizers',
                23: 'Other Chemical Products', 24: 'Plastics and Rubber',
                25: 'Logs and Wood in the Rough', 26: 'Wood Products', 27: 'Pulp, Paper, Paperboard',
                28: 'Paper or Paperboard Articles', 29: 'Printed Products', 30: 'Textiles and Leather',
                31: 'Non-Metallic Mineral Products', 32: 'Base Metal in Primary Forms', 
                33: 'Articles of Base Metal', 34: 'Machinery',
                35: 'Electronic and Electrical Equipment', 36: 'Motorized and Other Vehicles',
                37: 'Transportation Equipment', 38: 'Precision Instruments',
                39: 'Furniture and Lighting', 40: 'Miscellaneous Manufactured Products',
                41: 'Waste and Scrap', 43: 'Mixed Freight'
            }
            
            # Load flow data
            csv_file = csv_files[0]
            flows_df = pd.read_csv(f'results/{csv_file}')
            
            # Create network visualization
            fig, ax = plt.subplots(figsize=(14, 10))
            
            # Define state positions (triangle layout)
            positions = {
                'TX': (0.15, 0.15),   # Bottom left
                'CA': (0.85, 0.15),   # Bottom right  
                'NY': (0.5, 0.85)     # Top center
            }
            
            # Draw state nodes
            for state, pos in positions.items():
                circle = plt.Circle(pos, 0.06, color='darkgray', zorder=3)
                ax.add_patch(circle)
                ax.text(pos[0], pos[1], state, ha='center', va='center', 
                       fontsize=14, fontweight='bold', color='white', zorder=4)
            
            # Calculate flow values and top commodities for each route
            route_data = {}
            for (origin, dest), group in flows_df.groupby(['orig_state_name', 'dest_state_name']):
                total_value = group['weighted_value'].sum()
                # Find top commodity for this route
                top_commodity = group.loc[group['weighted_value'].idxmax()]
                sctg_code = top_commodity['SCTG']
                commodity_name = SCTG_CODES.get(sctg_code, f"SCTG-{sctg_code}")
                route_data[(origin, dest)] = {
                    'value': total_value,
                    'top_commodity': commodity_name,
                    'sctg_code': sctg_code
                }
            
            # Normalize arrow thickness
            max_flow = max([data['value'] for data in route_data.values()])
            
            # Draw arrows for each flow with commodity labels
            for (origin, dest), data in route_data.items():
                if origin in positions and dest in positions:
                    start_pos = positions[origin]
                    end_pos = positions[dest]
                    value = data['value']
                    commodity = data['top_commodity']
                    
                    # Calculate arrow thickness
                    thickness = (value / max_flow) * 0.008
                    
                    # Draw arrow with slight curve to avoid overlap
                    curve_direction = 0.15 if (origin, dest) < (dest, origin) else -0.15
                    arrow = patches.FancyArrowPatch(start_pos, end_pos,
                                                   connectionstyle=f"arc3,rad={curve_direction}",
                                                   arrowstyle='->', 
                                                   mutation_scale=15,
                                                   linewidth=thickness * 1200,
                                                   color='steelblue',
                                                   alpha=0.7,
                                                   zorder=2)
                    ax.add_patch(arrow)
                    
                    # Add value and commodity label for all flows (lowered threshold for complete triangle)
                    if value > max_flow * 0.05:  # Label flows >5% of max (shows all 6 routes)
                        # Calculate label position along the curved path
                        mid_x = (start_pos[0] + end_pos[0]) / 2
                        mid_y = (start_pos[1] + end_pos[1]) / 2
                        
                        # Offset label based on curve direction
                        label_offset_x = curve_direction * 0.08
                        label_offset_y = 0.04 if curve_direction > 0 else -0.04
                        
                        # Shorten commodity name for arrow labels
                        short_commodity = commodity[:15] + "..." if len(commodity) > 15 else commodity
                        
                        ax.text(mid_x + label_offset_x, mid_y + label_offset_y, 
                               f'${value/1e9:.1f}B\n{short_commodity}', 
                               ha='center', va='center', fontsize=8,
                               bbox=dict(boxstyle="round,pad=0.2", facecolor="white", alpha=0.9),
                               zorder=5)
            
            # Create commodity legend with proper names
            unique_commodities = {}
            for data in route_data.values():
                sctg = data['sctg_code']
                commodity = data['top_commodity']
                if sctg not in unique_commodities:
                    unique_commodities[sctg] = commodity
            
            # Build legend text with proper commodity names
            legend_text = "Commodity Types:\n"
            for sctg, commodity in sorted(unique_commodities.items()):
                # Shorten commodity names for legend
                short_name = commodity[:22] + "..." if len(commodity) > 22 else commodity
                legend_text += f"SCTG-{sctg}: {short_name}\n"
            
            # Set plot properties
            ax.set_xlim(0, 1)
            ax.set_ylim(0, 1)
            ax.set_aspect('equal')
            ax.axis('off')
            ax.set_title('Interstate Commerce Network: TX-CA-NY Triangle\nArrow thickness = Trade volume, Labels show top commodity per route', 
                        fontsize=14, pad=20)
            
            # Add usage legend (bottom left)
            ax.text(0.02, 0.02, 'Arrow thickness = Trade volume\nLabels: Value + Top commodity\nAll 6 routes labeled', 
                   transform=ax.transAxes, fontsize=10,
                   bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgray", alpha=0.8))
            
            # Add commodity legend (bottom right)
            ax.text(0.98, 0.02, legend_text.strip(), 
                   transform=ax.transAxes, fontsize=9, ha='right', va='bottom',
                   bbox=dict(boxstyle="round,pad=0.3", facecolor="lightyellow", alpha=0.8))
            
            plt.tight_layout()
            plt.show()
            
            print("Enhanced network visualization complete!")
            print(f"Total trade flows visualized: ${sum([data['value'] for data in route_data.values()])/1e9:.1f} billion")
            
            # Print route summary
            print("\nRoute Summary:")
            for (origin, dest), data in sorted(route_data.items(), key=lambda x: x[1]['value'], reverse=True):
                print(f"  {origin} → {dest}: ${data['value']/1e9:.1f}B (Top: {data['top_commodity']})")

## Citation Information

**Data Source:**
U.S. Census Bureau. (2017). Commodity Flow Survey Public Use File. 
Retrieved from https://www.census.gov/programs-surveys/cfs/data/datasets.html

**Analysis Code:**
Available at: https://github.com/rsthornton/cfs-network-analysis

**Method:**
Three-level network centrality analysis using NetworkX, with survey-weighted interstate commodity flows.